# Laboratorium 1 - content-based recommender

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from sklearn.model_selection import train_test_split, KFold

In [2]:
# tworzymy reprezentacje filmow jako wektorow cech - na podstawie gatunkow

genres = [
    '(no genres listed)', 
    'Action', 
    'Adventure', 
    'Animation', 
    'Children', 
    'Comedy', 
    'Crime', 
    'Documentary', 
    'Drama', 
    'Fantasy', 
    'Film-Noir', 
    'Horror', 
    'IMAX', 
    'Musical', 
    'Mystery', 
    'Romance', 
    'Sci-Fi', 
    'Thriller', 
    'War', 
    'Western'
]
genres_no = len(genres)

movies = pandas.read_csv('ml-latest-small/movies.csv')
movies_no = movies.shape[0]

movies['bias'] = 1.0
for genre in genres:
    movies[genre] = np.where(movies['genres'].str.contains(genre, regex=False), 1.0, 0.0)
    
movies = movies.drop(columns=['title', 'genres']).set_index('movieId')
movies

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# wczytujemy oceny uytkownikow i od razu dzielimy je na dwa zbiory - treningowy i testowy

all_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
train_ratings_set, test_ratings_set = train_test_split(all_ratings, test_size=0.05)
train_ratings_set

,userId,movieId,rating
20361,135,349,2.0
89047,576,1373,1.0
10317,66,8636,5.0
81789,517,6951,2.0
12704,81,10,1.0
...,...,...,...
22705,156,260,1.0
34183,232,6059,3.5
8290,57,1643,4.0
47911,309,6947,4.0


In [4]:
# inicjalizujemy macierz preferencji uzytkownikow liczbami losowymi z przedzialu [0.0, 5.0]

def initialize_users(raw_ratings):
    users_no = raw_ratings['userId'].unique().size
    users = pandas.DataFrame(5.0 * np.random.uniform(size=(users_no, genres_no+1)), index=raw_ratings['userId'].unique(), columns=['bias']+genres)
    return users_no, users

users_no, users = initialize_users(train_ratings_set)
users

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
135,4.143583,4.720884,4.484994,4.021148,1.488019,4.993692,1.456521,2.060269,3.628767,1.306733,...,3.167996,1.149016,2.639151,3.094787,2.471560,1.657343,4.575149,4.265469,0.860362,2.886523
576,4.542866,4.822788,4.691875,1.759111,1.296179,2.871898,0.589354,2.965890,3.743302,0.943092,...,4.404660,1.640940,1.803936,4.266215,1.120679,1.691305,1.834585,2.429630,0.293147,0.586022
66,1.203134,3.179880,0.736263,0.653414,0.432549,0.512862,1.261118,4.210701,2.984076,0.619723,...,1.700676,0.744247,2.211317,0.043885,1.420613,4.046095,2.248303,4.474488,4.135317,4.832562
517,2.532188,4.385041,0.668997,4.297517,4.268352,0.740792,3.988887,1.476885,4.184425,0.809756,...,0.603055,0.493950,1.616847,4.039712,0.763846,1.972039,0.367830,2.584635,3.788547,0.763863
81,2.631958,3.989208,1.774494,0.306407,2.452612,0.250900,1.798504,0.967781,0.925568,3.070404,...,4.512749,2.997137,4.946563,3.657504,0.962385,3.867642,4.297470,1.754569,3.878569,3.030747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,0.734832,0.460167,0.558310,0.995267,0.327845,4.647566,2.880388,3.392492,1.236209,3.475072,...,4.411656,1.365539,4.136613,4.952257,0.067456,3.349684,2.583346,2.299743,0.494574,1.583548
516,1.183022,4.693555,1.428361,0.058697,3.289316,4.357905,4.083981,2.549110,1.311496,1.196906,...,1.394740,1.260966,4.785844,1.871262,4.854064,2.337124,1.590458,4.002593,0.328699,1.163959
388,0.277283,3.318202,3.350592,1.140875,3.663687,1.901915,1.699719,3.599874,0.044237,3.538362,...,3.136363,2.755254,3.956632,4.316775,4.986066,2.061057,1.896182,1.099955,3.722088,1.757501
37,0.210743,2.065360,1.377591,1.452022,0.784436,3.645224,4.052670,0.395877,3.974126,3.689642,...,1.567487,4.641594,3.785644,0.378186,0.624004,3.352533,1.146938,4.162568,1.416608,2.620370


In [5]:
# za pomoca sprytnej sztuczki przeksztalcamy oceny z formatu dostarczonego przez MovieLens do uzytecznej macierzy
# zwroc uwage na to, ze czesci filmow moze brakowac po podziale datasetu na dwie czesci - musimy uzueplnic brakujace kolumny

def get_ratings(raw_ratings, movies):
    ratings = raw_ratings.pivot(*raw_ratings.columns).fillna(0.0)
    missing_movies = set(movies.index).difference(set(raw_ratings['movieId']))
    for movie in missing_movies:
        ratings[movie] = 0.0
    ratings = ratings.reindex(sorted(ratings.columns), axis=1)
    return ratings

ratings = get_ratings(train_ratings_set, movies)
ratings

C:\Users\lenovo\AppData\Local\Temp/ipykernel_8500/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Część 2. - trening modelu

In [17]:
# trenujemy model iteracyjnie, wykorzystujac gradient descent

# alpha = 0.0001 # learning speed
# delta = 5000 # minimal upgrade for each step
# lambd = 0.01 # regularization weight

alpha = 0.00005 # learning speed
delta = 25 # minimal upgrade for each step
lambd = 50 # regularization weight

def calculate_user_preferences(users, movies, ratings, raw_ratings, users_no, movies_no, alpha, delta, lambd):
    total_error = 0.0
    model = users
    counter = 0

    while(True):
        previous_total_error = total_error

        # mozemy to policzyc jako iloczyn skalarny preferencji uzytkownikow i cech filmow
        predicted_ratings = model.dot(movies.transpose())
#         predicted_ratings = movies.transpose() @ users 
        # tu stosujemy bardzo przydatna funkcje NumPy
        errors = np.where(ratings==0.0, pandas.DataFrame(np.zeros((users_no, movies_no))), predicted_ratings - ratings)
        # znow iloczyn skalarny - tym razem bledow
        gradient = errors.dot(movies)

        # tu stosujemy pewna sztuczke - rozbijamy sobie macierz z wyrazami regularyzujacymi na dwie
#         pierwsza to kolumna zlozona z zer
        regularization_k0 = pandas.DataFrame(np.zeros((users_no, 1)), index=raw_ratings['userId'].unique(), columns=['bias'])
        # druga to macierz preferencji uzytkownikow (czyli modelu) - bez pierwszej kolumny
        regularization_k = model.iloc[:,1:]
        
        # teraz sklejamy obie macierze
        regularization = pandas.concat([regularization_k0, regularization_k], axis=1)

#         # najwazniejszy krok - aktualizacja modelu, czyli wszystkich wag
        model = model - alpha * (gradient + lambd * regularization)

    # suma wszystkich bledow
    
        total_error = errors.sum() # suma elementow macierzy errors
        if math.isnan(total_error):
            break
        print("Total error = " + str(total_error))
        progress = abs(previous_total_error - total_error)
        print("Progress = " + str(progress))
        print("\n\n")
        counter += 1
        if progress < delta or counter > 100:
            break
            
    return model

# prediction_model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)

## Część 3. - ocena jakości algorytmu

In [ ]:
# https://stackoverflow.com/questions/16729574/how-to-get-a-value-from-a-cell-of-a-dataframe

In [7]:
def prepare_for_confusion_matrix(prediction_model, movies, test_ratings_set):
    predicted_ratings = prediction_model.dot(movies.T)
    predicted_ratings = predicted_ratings.astype(int)
    predicted_ratings = predicted_ratings.sort_index(0)

    y_actual = []
    y_predicted = []

    for index, row in test_ratings_set.iterrows():
        y_actual.append(row['rating'])
        y_predicted.append(predicted_ratings.at[int(row['userId']),int(row['movieId'])])

    return y_actual, y_predicted


y_actual, y_predicted = prepare_for_confusion_matrix(prediction_model, movies, test_ratings_set)


C:\Users\lenovo\AppData\Local\Temp/ipykernel_8500/3483175355.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  predicted_ratings = predicted_ratings.sort_index(0)


In [ ]:
# https://www.codegrepper.com/code-examples/python/sklearn+knn+example+confusion+matrix

In [ ]:
# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal

In [ ]:
# https://stats.stackexchange.com/questions/51296/how-do-you-calculate-precision-and-recall-for-multiclass-classification-using-co

In [8]:
# na podstawie zbioru testowego i wytrenowanego modelu obliczamy metryki opisujace jakosc modelu
## sklearn.confusion_matrix()

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

positive_threshold = 4.0
negative_threshold = 2.0

# def calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold):
def calculate_stats(y_actual, y_predicted, positive_threshold, negative_threshold):
    # obliczamy true_positives itp.
    # nastepnie wszystkie metryki
    
    y_actual_labels = []
    y_predicted_labels = []
    
    for i in range(0, len(y_actual)):
        if y_actual[i] > int(positive_threshold):
            label = 'Positive'
        elif y_actual[i] <= int(negative_threshold):
            label = 'Negative'
        else:
            label = 'None'
        y_actual_labels.append(label)
    
    for i in range(0, len(y_predicted)):
        if y_predicted[i] > positive_threshold:
            label = 'Positive'
        elif y_predicted[i] < negative_threshold:
            label = 'Negative'
        else:
            label = 'None'
        y_predicted_labels.append(label)
    
    conf_m = confusion_matrix(y_actual_labels, y_predicted_labels)
    
    false_positives = conf_m.sum(axis=0) - np.diag(conf_m)  
    false_negatives = conf_m.sum(axis=1) - np.diag(conf_m)
    true_positives = np.diag(conf_m)
    true_negatives = conf_m.sum() - (false_positives + false_negatives + true_positives)
    
    # fp, ... -> suma elementow w tablicy
    false_positives = false_positives.sum()
    false_negatives = false_negatives.sum()
    true_positives = true_positives.sum()
    true_negatives = true_negatives.sum()

#     print(precision_score(y_actual_labels, y_predicted_labels, average='micro')) # liczy recall, wynik taki sam jak ponizej

        
    recall = true_positives / (true_positives + false_negatives)
    precision = true_positives / (true_positives + false_positives)
    f1 =  2 * ((precision * recall) / (precision + recall))
    accuracy = (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives)

    return {
        'true_positives': true_positives,
        'true_negatives': true_negatives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [10]:
predicted_ratings = prediction_model.dot(movies.T)
calculate_stats(y_actual, y_predicted, positive_threshold, negative_threshold)

{'true_positives': 1422,
 'true_negatives': 6464,
 'false_positives': 3620,
 'false_negatives': 3620,
 'accuracy': 0.5213539600687558,
 'precision': 0.2820309401031337,
 'recall': 0.2820309401031337,
 'f1': 0.2820309401031337}

In [18]:
# dla porownania - obliczmy te same metryki dla modelu losowego
# zauwaz, w jaki sposob ponownie wykorzystujemy funkcje inicjalizujaca preferencje uzytkownikow

# tamten kod opakowac w funkcje zeby zroic tez dla random_prediction

_, random_model = initialize_users(train_ratings_set)
random_prediction = random_model.dot(movies.T)
print(random_prediction)
print("\n\n\n\n")
print(predicted_ratings)







# y_actual, y_predicted = prepare_for_confusion_matrix(prediction_model, movies, test_ratings_set)
y_actual_random, y_predicted_random = prepare_for_confusion_matrix(random_model, movies, train_ratings_set)
calculate_stats(y_actual_random, y_predicted_random, positive_threshold, negative_threshold)

movieId     1          2         3          4         5          6       \
135      13.832806   9.202388  4.124828   4.323169  3.822701  10.577689   
576      17.278679  10.791025  5.850018   7.564409  4.369493   5.087395   
66       11.367960   7.168443  5.761873   7.513494  1.272514  10.587524   
517      12.998203   8.253196  5.423392   8.288146  4.845891  11.924210   
81       17.289721  14.330537  6.605702  10.958281  4.869938  12.799462   
..             ...        ...       ...        ...       ...        ...   
531      18.918547  11.252506  6.784958   8.390714  5.017504   8.204151   
516      14.547090   9.385080  7.839042  11.505519  5.823622  12.877761   
388      15.815648  12.086220  8.414719  11.010216  7.537033   9.901604   
37       14.537717   6.774502  7.681050  10.824056  3.409474   8.247890   
194      16.571455   8.672397  7.382884  11.440272  6.235269   8.628825   

movieId    7         8         9          10      ...     193565     193567  \
135      4.124828  6

C:\Users\lenovo\AppData\Local\Temp/ipykernel_8500/3483175355.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  predicted_ratings = predicted_ratings.sort_index(0)


{'true_positives': 26331,
 'true_negatives': 122125,
 'false_positives': 69463,
 'false_negatives': 69463,
 'accuracy': 0.5165807183470085,
 'precision': 0.27487107752051276,
 'recall': 0.27487107752051276,
 'f1': 0.27487107752051276}

## Część 4. - istotność statystyczna

In [19]:
# wielokrotnie uruchamiamy trening modelu
# za każdym razem dzielimy dataset na zbior treningowy i testowy w inny sposob - klasa KFold robi to za nas
# zwroc uwage na bardzo istotny szczegol - oba modele, wytrenowany i losowy, musza byc porownywane na tym samym zbiorze testowym

n_tests = 5
positive_tests_count = 0
results = []
random_results = []
alpha = 0.01 # learning speed
delta = 500 # minimal upgrade for each step
lambd = 0.1 # regularization weight

# for test, train in KFold(n_splits=n_tests, shuffle=True).split(raw_ratings):
for train, test in KFold(n_splits=n_tests, shuffle=True).split(train_ratings_set):
    # wygeneruj macierz użytkowników i ocen
    # wytrenuj model
    # oblicz metryki dla wytrenowanego modelu
    # oblicz metryki dla modelu losowego
    users_no, users = initialize_users(train_ratings_set)
    ratings = get_ratings(train_ratings_set, movies)
#     ratings = get_ratings(raw_ratings, movies)
    model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)
    y_actual, y_predicted = prepare_for_confusion_matrix(model, movies, test_ratings_set)
    y_actual_random, y_predicted_random = prepare_for_confusion_matrix(random_model, movies, test_ratings_set)
    stats = calculate_stats(y_actual, y_predicted, positive_threshold, negative_threshold)
    random_stats = calculate_stats(y_actual_random, y_predicted_random, positive_threshold, negative_threshold)
    if stats['recall'] > random_stats['recall']:
        positive_tests_count += 1
    print("Recall for trained: " + str(stats['recall']))
    print("Recall for random: " + str(random_stats['recall']))
    print("\n\n\n\n")
    

C:\Users\lenovo\AppData\Local\Temp/ipykernel_8500/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


Total error = 553466.4949990283
Progress = 553466.4949990283



Total error = -922530.8073117444
Progress = 1475997.3023107727



Total error = 998004.9940895983
Progress = 1920535.8014013427



Total error = -1425569.0136677937
Progress = 2423574.007757392



Total error = 7194916.133366933
Progress = 8620485.147034727



Total error = -48092426.65110507
Progress = 55287342.784472



Total error = 496146027.2223083
Progress = 544238453.8734133



Total error = -4353073087.734815
Progress = 4849219114.957123



Total error = 40396994117.11161
Progress = 44750067204.84643



Total error = -372163534692.61926
Progress = 412560528809.7309



Total error = 3423633323936.0747
Progress = 3795796858628.694



Total error = -31547449631243.49
Progress = 34971082955179.562



Total error = 290486408288731.5
Progress = 322033857919975.0



Total error = -2675587072432821.5
Progress = 2966073480721553.0



Total error = 2.4641212127395316e+16
Progress = 2.7316799199828136e+16



Total error = -2.

C:\Users\lenovo\AppData\Local\Temp/ipykernel_8500/3483175355.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  predicted_ratings = predicted_ratings.sort_index(0)


Recall for trained: 0.1453788179293931
Recall for random: 0.26715589051963506







C:\Users\lenovo\AppData\Local\Temp/ipykernel_8500/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


...

Recall for trained: 0.1453788179293931
Recall for random: 0.26715589051963506







C:\Users\lenovo\AppData\Local\Temp/ipykernel_8500/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


Total error = 555381.9827758517
Progress = 555381.9827758517



Total error = -811645.8063207443
Progress = 1367027.7890965962



Total error = 1268903.0603600184
Progress = 2080548.8666807627



Total error = -3556666.1489812494
Progress = 4825569.209341268



Total error = 23082771.557737663
Progress = 26639437.706718914



Total error = -217047216.50765866
Progress = 240129988.0653963



Total error = 1990478690.3672113
Progress = 2207525906.87487



Total error = -18234579223.62289
Progress = 20225057913.9901



Total error = 168128699124.14392
Progress = 186363278347.7668



Total error = -1548122774064.3677
Progress = 1716251473188.5117



Total error = 14256769858769.248
Progress = 15804892632833.615



Total error = -131307192933956.83
Progress = 145563962792726.06



Total error = 1209328721933177.5
Progress = 1340635914867134.2



Total error = -1.1137777026546304e+16
Progress = 1.2347105748479482e+16



Total error = 1.0257837589583272e+17
Progress = 1.1371615292237902e+17



C:\Users\lenovo\AppData\Local\Temp/ipykernel_8500/3483175355.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  predicted_ratings = predicted_ratings.sort_index(0)


Recall for trained: 0.1453788179293931
Recall for random: 0.26715589051963506







In [20]:
# obliczamy, w ilu probach wytrenowany model okazal sie lepszy od losowego
# przeprowadzamy test statystyczny - jak prawdopodobne jest to, by k pozytywnych prob bylo dzielem przypadku

def possibility_of_at_least_k_successes_in_n(k, n):
    p = 0.0
    # obliczamy kolejno prawdopodobienstwo k sukcesow, k+1 sukcesow, ...
    # przydadza Ci sie funkcje marh.comb() i math.pow()
    for i in range(k, n):
        part = math.comb(n, k)
        part = part / (pow(2, n))
        p += part
    return p

p = 0.05
metric = 'recall'

print("Positive_test_counts = " + str(positive_tests_count))
print("Number of tests = " + str(n_tests))

# positive_tests_count =  # w ilu przypadkach okazalismy sie lepsi niz random?
if possibility_of_at_least_k_successes_in_n(positive_tests_count, n_tests) <= p:
    print('We are better than random!')
else:
    print('There is no evidence we are better')

Positive_test_counts = 0
Number of tests = 5
There is no evidence we are better
